In [1]:
import pandas as pd
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-py-m.us-central1-f.c.cncd-cncd.internal:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.59-63cf625e29e5
LOGGING: writing to /hail-20201215-0801-0.2.59-63cf625e29e5.log


In [2]:
!rm *.xlsx

rm: cannot remove '*.xlsx': No such file or directory


In [6]:
for chro in [16]:
#     try:#pgr-wgs-2k/VEP_VCF/Chr1_2k_wgs.mt
        print("Reading Data")
        data = hl.read_matrix_table("gs://pgr-wgs-8k/VEP_VCF/pgr8k_chr"+str(chro)+"_VEP.mt")
#         print("Filtering Transcripts")
#         canonical = data.vep.transcript_consequences.filter(lambda csq: csq.canonical == 1)
#         data = data.annotate_rows(canonical_transcript=canonical)
#         data = data.filter_rows( ( data.vep.transcript_consequences.gene_symbol.contains('GSDMD') ))
        data = data.filter_rows( ( data.vep.transcript_consequences.lof.contains('HC') ))
#         data = data.filter_rows(hl.len(data.canonical_transcript.exon) != 0)
#         data.rows().show()
        print(data.count())
#         print("Annotating DB")
#         db = hl.experimental.DB()
#         data = db.annotate_rows_db(data, 'clinvar_variant_summary','dbNSFP_genes','dbNSFP_variants','gnomad_exome_sites')
#         data.rows().show()
#         print("Saving to Disk")
        mt=data
#             mt.GT.show(10, n_cols=10)
        ab = mt.AD[1] / hl.sum(mt.AD)
        filter_condition_ab = ( (mt.GT.is_hom_ref() & (ab <= 0.1)) |
                                (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                                (mt.GT.is_hom_var() & (ab >= 0.9)))

        fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
#         print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
        mt1 = mt.filter_entries(filter_condition_ab)
#             mt1.GT.show(10, n_cols=10)

        filter_condition_ab = ( (mt1.GQ > 20) & (mt1.DP < 200) & (mt1.DP > 10) )
#             mt1.DP.show(10, n_cols=10)
        fraction_filtered = mt1.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
#         print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')

        mt3 = mt1.filter_entries(filter_condition_ab)
#             mt3.DP.show(10, n_cols=10)
        data = mt3
        
        data = data.filter_rows(data.variant_qc.AF[1] < 0.05)
        
        print('Annotate Heterozygous Alleles')
        ds_result1 = data.annotate_rows(hets = hl.agg.filter(data.GT.is_het(),hl.agg.collect(data.s)))
        ds_result1.count()
        print('Annotate N Hets')
        ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
        ds_result4.count()
        print('Annotate Homozygous Alleles')
        ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
        ds_result3.count()
        print('Annotating N Homs')
        ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))
        ds_result5.count()
#         ds_result5.rows().show()

        print('Selecting Data To Query')
        ds_result6 = ds_result5.rows()
        ds_result6 = ds_result6.key_by()

        print('Converting To Pandas')
        finData_All = ds_result6.select(
            locus=ds_result6.locus,
            alleles=ds_result6.alleles,
            gene=ds_result6.vep.transcript_consequences.gene_symbol,
            maf=ds_result6.variant_qc.AF[1],
            lof=ds_result6.vep.transcript_consequences.lof,
            consequence=ds_result6.vep.most_severe_consequence,
#             amino_acid=ds_result6.vep.transcript_consequences.amino_acids,
#             exons=ds_result6.vep.transcript_consequences.exon,
            amino_acids=ds_result6.vep.transcript_consequences.amino_acids,
            protein_end=ds_result6.vep.transcript_consequences.protein_end, 
            protein_start=ds_result6.vep.transcript_consequences.protein_start, 
            protein_id=ds_result6.vep.transcript_consequences.protein_id, 
            hets=ds_result6.hets,
            n_hets=ds_result6.n_het,
            homs=ds_result6.homs,
            n_homs=ds_result6.n_hom
        ).to_pandas()      

        finData_All.to_excel("Chr"+str(chro)+"8k_HC.xlsx")
        !gsutil -m cp *.xlsx gs://pgr-wgs-8k/Complete_Annotations/HC/
        !rm *.xlsx
        print("Done")

#         data.write("gs://pgr-wgs-2k/Complete_Annotations/Canonical/2K_Chr"+str(chro)+"_VEP_DB_C.mt", overwrite=True)
#     except:
#         print("Error in Chr", chro)

Reading Data
(26263, 7482)
Annotate Heterozygous Alleles
Annotate N Hets
Annotate Homozygous Alleles
Annotating N Homs
Selecting Data To Query
Converting To Pandas
Copying file://Chr168k_HC.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1/1 files][  2.1 MiB/  2.1 MiB] 100% Done                                    
Operation completed over 1 objects/2.1 MiB.                                      
Done


In [5]:
data.vep.transcript_consequences.lof.show()

,,
locus,alleles,
locus<GRCh38>,array<str>,array<str>
chr16:47439,"[""C"",""A""]","[NA,NA]"
chr16:47440,"[""C"",""A""]","[NA,NA]"
chr16:47443,"[""C"",""T""]","[NA,NA]"
chr16:47444,"[""G"",""A""]","[NA,NA]"
chr16:47447,"[""G"",""A""]","[NA,NA]"
chr16:47461,"[""T"",""C""]","[NA,NA]"
chr16:47462,"[""T"",""C""]","[NA,NA]"
chr16:47464,"[""C"",""T""]","[NA,NA]"


In [ ]:
ds_result5.info.ANN[0][0].show()

In [ ]:
for chro in list(range(1,23))+['X','Y']:
#     try:#pgr-wgs-2k/VEP_VCF/Chr1_2k_wgs.mt
        print("Reading Data")
        data = hl.read_matrix_table("gs://pgr-wgs-2k/VEP_VCF/Chr"+str(chro)+"_2k_wgs.mt")
        print(data.count())
        print("Filtering Transcripts")
        canonical = data.vep.transcript_consequences.filter( lambda csq: csq.canonical == 1 )
        data = data.annotate_rows(canonical_transcript=canonical)
#         print(data.count())
        data = data.canonical_transcript.filter( lambda csq: csq.exon != 0 )
        print(data.count())
#         data.canonical_transcript.exon.show(500)
#     except:
#         print("Error",chro)

In [ ]:
data.canonical_transcript.exon.show()

In [ ]:
# data = hl.read_matrix_table("gs://pgr-wgs-8k/VEP_VCF/pgr8k_chr21_VEP.mt")
# data.rows().show()
# 
# canonical = data.vep.transcript_consequences.filter(lambda csq: csq.canonical == 1)
data1 = data.annotate_rows(transcript_new=canonical)
data1.rows().show()
# canonical[1].show(1)


In [ ]:
chrList = [11]
geneList = ['SCN10A']

In [ ]:
for chro,gene in zip(chrList, geneList):
    try:
#         for database in [
# #             "gs://pgr-wes-r20/VEP_VCF/R20_Chr" + str(chro) + "_VEP.mt", 
# #                          "gs://pgr-wgs-2k/VEP_VCF/Chr" + str(chro) + "_2k_wgs.mt"
#                          "gs://pgr-wes-30k-mt/Complete_Annotations/Promis30_chr" + str(chro) + "_VEP_DB.mt"
#           ]:
            print('Importing Data')
            data2 = hl.read_matrix_table("gs://pgr-wgs-8k/VEP_VCF/pgr8k_chr11_VEP.mt")
            
    #         data2 = hl.read_matrix_table("gs://pgr-wgs-2k/VEP_VCF/Chr" + str(chro) + "_2k_wgs.mt")
    #         data2.rows().show()

#             data2 = data2.filter_rows( data2.vep.transcript_consequences.gene_symbol.contains(str(gene)) )
#             data2 = hl.split_multi_hts(data2)
            data2 = data2.filter_rows( (data2.vep.transcript_consequences.lof.contains('HC')) )
            print(data2.count())

#             mt=data2
# #             mt.GT.show(10, n_cols=10)
#             ab = mt.AD[1] / hl.sum(mt.AD)
#             filter_condition_ab = ( (mt.GT.is_hom_ref() & (ab <= 0.1)) |
#                                     (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
#                                     (mt.GT.is_hom_var() & (ab >= 0.9)))

#             fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
#             print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
#             mt1 = mt.filter_entries(filter_condition_ab)
# #             mt1.GT.show(10, n_cols=10)

#             filter_condition_ab = ( (mt1.GQ > 20) & (mt1.DP < 200) & (mt1.DP > 10) )
# #             mt1.DP.show(10, n_cols=10)
#             fraction_filtered = mt1.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
#             print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')

#             mt3 = mt1.filter_entries(filter_condition_ab)
# #             mt3.DP.show(10, n_cols=10)
#             data2 = mt3
#             print(data2.count())       

#             print('Annotate Heterozygous Alleles')
            ds_result1 = data2.annotate_rows(hets = hl.agg.filter(data2.GT.is_het(),hl.agg.collect(data2.s)))
            ds_result1.count()
#             print('Annotate N Hets')
            ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
            ds_result4.count()
#             print('Annotate Homozygous Alleles')
            ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
            ds_result3.count()
#             print('Annotating N Homs')
            ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))
            ds_result5.count()
#             ds_result5.vep.transcript_consequences.gene_symbol.show()
#             ds_result5 = hl.variant_qc(ds_result5)
            for maf in [0.05]:
                ds_result5 = ds_result5.filter_rows(ds_result5.variant_qc.AF[1] < maf)
                print()
                print('MAF <', maf, ds_result5.count())

#                 db = hl.experimental.DB()
#                 ds_result5 = db.annotate_rows_db(ds_result5, 'clinvar_variant_summary','dbNSFP_genes','dbNSFP_variants','gnomad_exome_sites')
#                 ds_result5.rows().show()

#                 print('Selecting Data To Query')
                ds_result6 = ds_result5.rows()
                ds_result6 = ds_result6.key_by()

                print('Converting To Pandas')
                finData_All = ds_result6.select(
                    locus=ds_result6.locus,
                    alleles=ds_result6.alleles,
                    gene=ds_result6.vep.transcript_consequences.gene_symbol,
                    lof=ds_result6.vep.transcript_consequences.lof,
                    consequence=ds_result6.vep.most_severe_consequence,
                    AF=ds_result6.variant_qc.AF[1],
                    hets=ds_result6.hets,
                    n_hets=ds_result6.n_het,
                    homs=ds_result6.homs,
                    n_homs=ds_result6.n_hom,
                    amino_acids=ds_result6.vep.transcript_consequences.amino_acids,
                    protein_end=ds_result6.vep.transcript_consequences.protein_end, 
                    protein_start=ds_result6.vep.transcript_consequences.protein_start, 
                    protein_id=ds_result6.vep.transcript_consequences.protein_id, 
    #                 exon=ds_result6.vep.transcript_consequences[0].exon,    
    #                 polyphen_prediction=ds_result6.vep.transcript_consequences[0].polyphen_prediction,
    #                 polyphen_score=ds_result6.vep.transcript_consequences[0].polyphen_score,
    #                 sift_prediction=ds_result6.vep.transcript_consequences[0].sift_prediction,
    #                 sift_score=ds_result6.vep.transcript_consequences[0].sift_score,
    #                 clinvar_signif=ds_result6.dbNSFP_variants.clinvar_clnsig,
    #                 clinvar_trait=ds_result6.dbNSFP_variants.clinvar_trait,
    #                 clinvar_MedGen_id=ds_result6.dbNSFP_variants.clinvar_MedGen_id,
    #                 REVEL_score=ds_result6.dbNSFP_variants.REVEL_score,
    #                 REVEL_rankscore=ds_result6.dbNSFP_variants.REVEL_rankscore,
    #                 MutationTaster_pred=ds_result6.dbNSFP_variants.MutationTaster_pred,
    #                 LRT_pred=ds_result6.dbNSFP_variants.LRT_pred,
    #                 Polyphen2_HVAR_pred=ds_result6.dbNSFP_variants.Polyphen2_HVAR_pred,
    #                 Polyphen2_HDIV_pred=ds_result6.dbNSFP_variants.Polyphen2_HDIV_pred,
    #                 CADD_phred=ds_result6.dbNSFP_variants.CADD_phred,
    #                 DANN_score=ds_result6.dbNSFP_variants.DANN_score
                ).to_pandas()       

                finData_All.to_excel('8K_Chr'+str(chro)+'.xlsx')
                !gsutil -m cp *.xlsx gs://pgr-wgs-8k/Output/
                !rm *.xlsx
                print("Done")
 
    except Exception as e: print(e)  #print("Error in Chr", str(chro)) 

In [ ]:
!gsutil ls gs://pgr-wgs-8k/Output/

In [ ]:
finData_All.head()

In [ ]:
data2 = hl.read_matrix_table("gs://pgr-wgs-8k/VEP_VCF/pgr8k_chr21_VEP.mt")

In [ ]:
print(data2.count())

In [ ]:
for chrNum in [21]: #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]:

#     pgr-wes-30k-mt/Vep_MT/Promis30_chr1_VEP.mt
    
    print("Reading Mt Chr", chrNum)
    data = hl.read_matrix_table("gs://pgr-wgs-8k/VEP_VCF/pgr8k_chr21_VEP.mt")  
    data.rows().show()
    print("Initial Count", data.count())
    
    #1- Splt Multi Allelic Variants
#     data1 = hl.split_multi_hts(data)
#     print('DONE MAV')
#     data1.rows().show()
#     print("After MAV Count")
#     print(data1.count())

#     db = hl.experimental.DB()
#     data1 = db.annotate_rows_db(data1, 'dbNSFP_genes','dbNSFP_variants','gnomad_exome_sites')
    
    #2- Remove variants in LCR, segDup, Decoy regions
    # Import gnomad LCR
    print('Reading LCR List')
    interval_table_lcr = hl.import_bed('gs://test1haseeb/LCR1.bed',
                                               reference_genome='GRCh37')
    print('Filtering LCR')
    data1 = data1.filter_rows(hl.is_defined(interval_table_lcr[data1.locus]) ,keep=False)
    print('Remove Variants in Lcr', data1.count())

    # Import gnomad SegDup
    print()
    print('Reading SegDup List')
    interval_table_segdup = hl.import_bed('gs://test1haseeb/intervals_segdup_hg19_self_chain_split_both.bed',
                                          reference_genome='GRCh37')
    print('Filtering SegDup')
    data2 = data1.filter_rows(hl.is_defined(interval_table_segdup[data1.locus]) ,keep=False)
    print('Remove Variants in SegDup', data2.count())

    # Import gnomad Decoy
    print()
    print('Reading Decoy List')
    interval_table_decoy = hl.import_bed('gs://test1haseeb/intervals_decoy_mm-2-merged.GRCh37_compliant.bed',
                                          reference_genome='GRCh37')
    print('Filtering Decoy')
    data3 = data2.filter_rows(hl.is_defined(interval_table_decoy[data2.locus]) ,keep=False)
    print('Remove Variants in Decoy', data3.count())
    data3.rows().show()
    
    
    data5 = hl.variant_qc(data3)
    data5.rows().show()
    
    #3- Genotype QC by Allele-Depth
    mt=data5
    ab = mt.AD[1] / hl.sum(mt.AD)
    filter_condition_ab = ( (mt.GT.is_hom_ref() & (ab <= 0.1)) |
                            (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                            (mt.GT.is_hom_var() & (ab >= 0.9)))
    print("3")
    fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
    print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
    mt1 = mt.filter_entries(filter_condition_ab)
#     mt1.GT.show(1, n_cols=10)
    
    #4- Filter variants with (GQ < 20), (DP < 10) or (DP > 200)
    filter_condition_ab = ( (mt1.GQ > 20) & (mt1.DP < 200) & (mt1.DP > 10) )

    fraction_filtered = mt1.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
    print('4')
    print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
    mt3 = mt1.filter_entries(filter_condition_ab)
#     mt3.GQ.show(5, n_cols=50)
    
    #5- Remove sites with missing more than 10% after above filtering.
    data1 = mt3.filter_rows(mt3.variant_qc.call_rate > 0.90)
    print('5', data1.count())
    
    #6- Remove sites with population specific HWE P value < 1e-8
    mt5 = data1.filter_rows(data1.variant_qc.p_value_hwe > 1e-8)
    print('6',mt5.count())
    mt5.rows().show()

    #7- Remove INDELS where QD <= 3
    filter_condition_ab = ( (hl.is_indel(mt5.alleles[0], mt5.alleles[1]) ) & (mt5.info.QD <= 3) )
    data1 = mt5.filter_rows(filter_condition_ab, keep=False)
    print('7',data1.count())
    data1.rows().show()

    #8- Remove SNPs where QD <= 2
    filter_condition_ab = ( (data1.info.QD <= 2) )
    data2 = data1.filter_rows(filter_condition_ab, keep=False)
    print('8', data2.count())
    data2.rows().show()   
    
    #9- Remove variants with InbreedingCoeff <= -0.3
    filter_condition_ab = ( (data2.info.InbreedingCoeff <= -0.3) )
    data3 = data2.filter_rows(filter_condition_ab, keep=False)
    print('9', data3.count())
    data3.rows().show()
    
    #10- Remove variants where VQSR does not PASS unless singleton in which case retain variants with VQSRTrancheSNP99.60to99.80
    pass_ds = data3.filter_rows( (hl.len(data3.filters) == 0) | (data3.filters.contains('VQSRTrancheSNP99.60to99.80')) )
    print('10', pass_ds.count())
#     pass_ds1 = pass_ds.filter_rows( (pass_ds.filters.contains('VQSRTrancheSNP99.60to99.80')) &
#                                 (pass_ds.gnomad_exome_sites.singleton != True),
#                                  keep=False)

    #11- Remove MAF
    pass_ds = pass_ds.filter_rows(pass_ds.variant_qc.AF[1] < 0.05)


    
    pass_ds.write('gs://pgr-wgs-2k/QC/Chr' + str(chrNum) + '_2k_QC.mt', overwrite=True)
    print("Final Count")
    print(pass_ds.count())